In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
ecommerce=pd.read_csv('Ecommerce_Delivery_Analytics_New.csv')
ecommerce

,Order ID,Customer ID,Platform,Order Date & Time,Delivery Time (Minutes),Product Category,Order Value (INR),Customer Feedback,Service Rating,Delivery Delay,Refund Requested
0,ORD000001,CUST2824,JioMart,19:29.5,30,Fruits & Vegetables,382,"Fast delivery, great service!",5,No,No
1,ORD000002,CUST1409,Blinkit,54:29.5,16,Dairy,279,Quick and reliable!,5,No,No
2,ORD000003,CUST5506,JioMart,21:29.5,25,Beverages,599,Items missing from order.,2,No,Yes
3,ORD000004,CUST5012,JioMart,19:29.5,42,Beverages,946,Items missing from order.,2,Yes,Yes
4,ORD000005,CUST4657,Blinkit,49:29.5,30,Beverages,334,"Fast delivery, great service!",5,No,No
...,...,...,...,...,...,...,...,...,...,...,...
99995,ORD099996,CUST5324,JioMart,49:29.5,24,Dairy,289,Packaging could be better.,3,No,No
99996,ORD099997,CUST1677,JioMart,18:29.5,19,Snacks,322,Good quality products.,4,No,No
99997,ORD099998,CUST8198,JioMart,27:29.5,41,Dairy,135,"Fast delivery, great service!",5,Yes,No
99998,ORD099999,CUST9975,JioMart,14:29.5,31,Grocery,973,Quick and reliable!,5,No,No


In [12]:
# ============================================
# Data Cleaning & Preprocessing
# ============================================
# 1. Identified missing values in the dataframe.

ecommerce.isnull().sum()


Order ID                   0
Customer ID                0
Platform                   0
Order Date & Time          0
Delivery Time (Minutes)    0
Product Category           0
Order Value (INR)          0
Customer Feedback          0
Service Rating             0
Delivery Delay             0
Refund Requested           0
dtype: int64

In [23]:
#2. Identified Duplicate values in the dataframe.
#----------------------------------------------------
# inspect the  duplicate rows and columns to ensure data integrity.
# No duplicate values were found.

ecommerce.duplicated().sum()
ecommerce.columns

Index(['Order ID', 'Customer ID', 'Platform', 'Order Date & Time',
       'Delivery Time (Minutes)', 'Product Category', 'Order Value (INR)',
       'Customer Feedback', 'Service Rating', 'Delivery Delay',
       'Refund Requested'],
      dtype='object')

In [48]:
# 3.Are there inconsistent formats?
# --------------------------------------------
# Standardized text fields by converting to lowercase and stripping whitespace.
# Corrected inconsistent entries if any 'product_category_name','Platform'.

ecommerce['Product Category']=ecommerce['Product Category'].str.strip()
ecommerce['Platform']=ecommerce['Platform'].str.strip()
ecommerce[['Product Category','Platform']]

,Product Category,Platform
0,fruits & vegetables,jiomart
1,dairy,blinkit
2,beverages,jiomart
3,beverages,jiomart
4,beverages,blinkit
...,...,...
99995,dairy,jiomart
99996,snacks,jiomart
99997,dairy,jiomart
99998,grocery,jiomart


In [35]:
# ============================================
# Data Manipulation 
# ============================================
# 4. calculate average delivery time, refund rate, and service rating for the orders
# Grouping the series 'Platform','Delivery Time (Minutes)','Refund Requested','Service Rating'.
# calculating the platform wise orders avg delivery time, requested the refund and the service rating

ecommerce.groupby('Platform').agg(
    Orders=('Order ID','count'),
    Avg_Delivery_Time=('Delivery Time (Minutes)','mean'),
    Refunds=('Refund Requested',lambda x: (x=='Yes').sum()),
    Avg_Service_Rating=('Service Rating', 'mean')
).reset_index()

,Platform,Orders,Avg_Delivery_Time,Refunds,Avg_Service_Rating
0,blinkit,33424,29.474898,15353,3.233844
1,jiomart,33127,29.634498,15178,3.245147
2,swiggy instamart,33449,29.499925,15288,3.243415


In [43]:
# 5.Create a summary table: Orders, delays, refunds, and average rating per product category
# calculating the Delays with the values 'Yes/No' by adding them
# calculating the Refund Requested with the values 'Yes/No' by adding them

summary_df = ecommerce.groupby('Product Category').agg(
    Orders=('Order ID', 'count'),
    Delays=('Delivery Delay', lambda x: (x == 'Yes').sum()),
    Refunds=('Refund Requested', lambda x: (x == 'Yes').sum()),
    Avg_Rating=('Service Rating', 'mean')
).reset_index()

summary_df

,Product Category,Orders,Delays,Refunds,Avg_Rating
0,beverages,16536,2236,7549,3.247581
1,dairy,16857,2312,7691,3.242688
2,fruits & vegetables,16632,2290,7692,3.230038
3,grocery,16737,2313,7811,3.217841
4,personal care,16533,2254,7517,3.253130
5,snacks,16705,2267,7559,3.253637


In [50]:
# ============================================
# Aggregation & Summarization Insights
# ============================================
# 6. Which platform has the highest average service rating?
# Calculating the Avg service rating by grouping with platform

ecommerce.groupby('Platform')['Service Rating'].mean().reset_index(name='Avg_Service_Rating').sort_values(by='Avg_Service_Rating', ascending=True)

,Platform,Avg_Service_Rating
0,blinkit,3.233844
2,swiggy instamart,3.243415
1,jiomart,3.245147


In [60]:
# 7. Which product category has the most refund requests?
# Grouping by Product category and calculating the Refund request per product
# calculating the Refund status based on there values 'Yes/No'

ecommerce.groupby('Product Category').agg(
    Customers=('Customer ID','count'),
    Refunds_Requested=('Refund Requested', lambda x: (x=='Yes').sum()),
    Delivered=('Refund Requested', lambda x: (x=='No').sum())
).reset_index()

,Product Category,Customers,Refunds_Requested,Delivered
0,beverages,16536,7549,8987
1,dairy,16857,7691,9166
2,fruits & vegetables,16632,7692,8940
3,grocery,16737,7811,8926
4,personal care,16533,7517,9016
5,snacks,16705,7559,9146


In [13]:
# 8. What percentage of total orders were delayed?
# Calculating the 'Delivery Delay' status with there values 'Yes/No'

ecommerce_delay=len(ecommerce[ecommerce['Delivery Delay']=='Yes'])
ecommerce_delay/ecommerce.shape[0]*100

13.672